In [1]:
pip install tensorflow==2.4.0

     |████████████████████████████████| 394.7MB 41kB/s 
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

     |████████████████████████████████| 14.9MB 212kB/s 
     |████████████████████████████████| 2.1MB 50.2MB/s 
     |████████████████████████████████| 11.6MB 25.7MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=00150bb4b78854e6ea3f97c2a788d9877836aa0264a40932b22ab8f4e51d5a68
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-resnet
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bokeh 2.3.1 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
  

In [3]:
pip install imageai --upgrade

     |████████████████████████████████| 163kB 9.0MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!pip install jsonlines

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/
!ls

In [ ]:
import json
import jsonlines
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
#!unzip '/content/drive/My Drive/Colab Notebooks/aca2021-meme-analysis-challenge.zip'

In [ ]:
import json
import jsonlines
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
with jsonlines.open('train.jsonl') as reader:
    data = pd.DataFrame(reader)

In [ ]:
data

In [ ]:
import os
directory = '/content/drive/My Drive/Colab Notebooks/img'

In [ ]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()

detector.setModelPath( os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/resnet50_coco_best_v2.1.0.h5"))
detector.loadModel()

object_list_tr = []

a = 1

for rindex, row in data.iterrows():
    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/{}".format(row['img'])), output_image_path=os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/kk.png"))

    text = ' '

    for eachObject in detections:
        
        text = text + ' '+ eachObject["name"]
    object_list_tr.append(text) 
    row['text'] = row['text']+text
    a += 1
    print(a)


In [ ]:
import csv
import pandas as pd

data.to_csv('/content/drive/My Drive/Colab Notebooks/tr.csv')

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.2, random_state = 42)

In [ ]:
!pip install transformers

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig
import tensorflow as tf

In [ ]:
from transformers import DistilBertTokenizer
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

In [ ]:
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=128, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')

In [ ]:
input_id, input_mask, _ = tokenize(X_train, tokenizer)

In [ ]:
input_id_all, input_mask_all, _ = tokenize(data["text"], tokenizer)


In [ ]:
distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(embedding_layer)
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dropout(0.25)(X)
X = tf.keras.layers.Dense(150, activation='relu')(X)
X = tf.keras.layers.Dropout(0.25)(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.5)(X)
X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
  layer.trainable = False

In [ ]:
N_EPOCHS = 8
BATCH_SIZE = 32

optimizer = tf.keras.optimizers.Adam(learning_rate= 2.5e-3)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', tf.keras.metrics.AUC()])

model.fit([input_id, input_mask], y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, validation_split = 0.2)

In [ ]:
input_id_test, input_mask_test, _ = tokenize(X_test, tokenizer)

In [ ]:
y_pred = model.predict([input_id_test, input_mask_test])

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
roc_auc_score(y_test, y_pred)

In [ ]:
with jsonlines.open('/content/drive/My Drive/Colab Notebooks/test.jsonl') as reader:
    data_test = pd.DataFrame(reader)

In [ ]:
data_test

In [ ]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()

detector.setModelPath( os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/resnet50_coco_best_v2.1.0.h5"))
detector.loadModel()

object_list_ts = []

a = 1

for rindex, row in data_test.iterrows():
    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/{}".format(row['img'])), output_image_path=os.path.join(execution_path , "/content/drive/My Drive/Colab Notebooks/kk.png"))

    text = ' '

    for eachObject in detections:
        
        text = text + ' '+ eachObject["name"] 
    object_list_ts.append(text)
    row['text'] = row['text']+text
    a += 1
    print(a)

In [ ]:
import csv
import pandas as pd

data_test.to_csv('/content/drive/My Drive/Colab Notebooks/ts.csv')

In [ ]:
input_id_test, input_mask_test, _ = tokenize(data_test['text'], tokenizer)

In [ ]:
y_pred = model.predict([input_id_test, input_mask_test])

In [ ]:
Submission = pd.DataFrame()
Submission['id'] = data_test['id']
Submission['label'] = y_pred
Submission.to_csv("/content/drive/My Drive/Colab Notebooks/res_all_data_5_32_0.2.csv",index=False)